# Install libraries

In [1]:
!pip3 install pyspark
!pip3 install OTXv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 4.9 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425360 sha256=5cdf76bf1d875e7e55e55f21654d69bbb0f88cf203360ed86b6e656c216434d6
  Stored in directory: /Users/joaopino/Library/Caches/pip/wheels/57/bd/14/ce9e21f2649298678d011fb8f71ed38ee70b42b94fef0be142
Successfully built pyspark

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip


In [2]:
!pip3 install httpx

  Obtaining dependency information for httpx from https://files.pythonhosted.org/packages/a2/65/6940eeb21dcb2953778a6895281c179efd9100463ff08cb6232bb6480da7/httpx-0.25.2-py3-none-any.whl.metadata
  Obtaining dependency information for anyio from https://files.pythonhosted.org/packages/bf/cd/d6d9bb1dadf73e7af02d18225cbd2c93f8552e13130484f1c8dcfece292b/anyio-4.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpcore==1.* from https://files.pythonhosted.org/packages/56/ba/78b0a99c4da0ff8b0f59defa2f13ca4668189b134bd9840b6202a93d9a0f/httpcore-1.0.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 1.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 2.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To updat

# Imports

In [1]:
# Access and see the db file
import sqlite3
import pandas as pd
from pandas import json_normalize
# PySpark
import pyspark
from pyspark.sql import SparkSession
# Drive
from OTXv2 import  OTXv2, IndicatorTypes

import json

# Get all the pulses

In [4]:
key = "b33bf61f62c7e7da86bb84481591a0844630323831ec210fd8c2e2efbfdc131a"
otx = OTXv2(key)

pulses = otx.getall()

In [5]:
json_normalize(pulses[1]["indicators"][0])

,id,indicator,type,created,content,title,description,expiration,is_active,role
0,2324419,9f177a6fb4ea5af876ef8a0bf954e37544917d9aaba046...,FileHash-SHA256,2023-12-06T17:05:17,,Armadillov1xxv2xx,SHA256 of 9a5fa5c5f3915b2297a1c379be9979f0,None,1,None


In [6]:
from pandas import json_normalize
# Get indicator IDs, names and created_dates
indicator_ids = list()
indicator_names = list()
created_date_list = list()

for pulse in pulses:
    for indicator in pulse["indicators"]:
        if indicator["type"] == "IPv4":
            indicator_ids.append(indicator["id"])
            created_date_list.append(indicator["created"])
            indicator_names.append(indicator["indicator"])
        
import gc
del pulses
gc.collect()

# Get all the indicator ids and created dates
print(len(indicator_ids))
print(len(created_date_list))
print(len(indicator_names))

3557
3557
3557


In [7]:
print(created_date_list[0:5])

['2023-12-06T17:04:52', '2023-12-06T17:04:52', '2023-12-06T17:04:52', '2023-12-06T17:04:52', '2023-12-06T17:04:52']


In [ ]:
# Pre-process the created date list
for i in range(len(created_date_list)):
    previous = created_date_list[i][:9]
    previous.replace("-","")
    print(previous)
    

In [9]:
from tqdm import tqdm

countries = list()
for i in tqdm(range(0, len(indicator_ids))):
    
    try :
        indicator_details = otx.get_indicator_details_full(IndicatorTypes.IPv4, indicator_names[i])
    except:
        print("Didn't work")
        countries.append("NULL")
    
    try: 
        countries.append(indicator_details["geo"]["country_name"])
    except:
        print("Didn't work")
        countries.append("NULL")

print(len(countries))

  3%|▎         | 90/3557 [01:51<1:44:42,  1.81s/it]

Didn't work


  4%|▍         | 139/3557 [04:12<2:35:04,  2.72s/it]

Didn't work


  7%|▋         | 240/3557 [08:13<2:54:54,  3.16s/it]

...

 53%|█████▎    | 1883/3557 [45:08<43:54,  1.57s/it]  

Didn't work


 64%|██████▍   | 2284/3557 [1:01:58<3:59:37, 11.29s/it]

Didn't work


 66%|██████▌   | 2340/3557 [1:04:01<42:20,  2.09s/it]  

In [ ]:
from pyspark.sql import SparkSession 

spark = SparkSession.builder.appName('sparkdf').getOrCreate() 

columns = ["Indicator_IDs", "Indicator", "Created Date", "Countries"]   
data = [indicator_ids, created_date_list, indicator_names, countries] 

# creating a dataframe 
dataframe = spark.createDataFrame(data, columns) 
dataframe.show() 

df = dataframe.withColumn("creation_date", col("creation_date").cast("date"))

# Group by day and country, count occurrences, and order by day and country
result_df = (df.groupBy(date_format("creation_date", "yyyy-MM-dd").alias("Day"), "countries")
              .agg(count("*").alias("Count"))
              .orderBy("Day", "countries"))

result_df.show()




In [ ]:
'''
Next steps:
3. Put it through spark 
'''

### Schema

In [ ]:
df.printSchema()